In [2]:
import requests
import lxml.html as lh
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

<b> 1. Extract HTML elements from page to here for preparation, use them to create three lists, each will represent a column in the dataframe</b>

In [209]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
soup = BeautifulSoup(page.content,'html.parser')
tb = soup.find('table',class_="wikitable sortable")
#rows = tb.find_all('tr')
cols= tb.find_all('td')

#====================create first version of each column=================
i=0;
Postcode=[]
Borough=[]
Neighbourhood=[]
while i < len(cols):
    Postcode.append(cols[i])
    Borough.append(cols[i+1])
    Neighbourhood.append(cols[i+2])
    i=i+3

#===========================clean the lists==================================
Postcode_fin=[]
Borough_fin=[]
Neighbourhood_fin=[]
#scan thru the lines in list Postcode, only keep the text in each line 
for code in Postcode:
    Postcode_fin.append(code.get_text())

for code in Borough:
    Borough_fin.append(code.get_text())

for code in Neighbourhood:
    line=code.get_text()
    Neighbourhood_fin.append(line.rsplit('\n')[0])
#================put together the dataframe ================================
dict = {"Postcode":Postcode_fin,"Borough":Borough_fin,"Neighbourhood": Neighbourhood_fin}
df = pd.DataFrame(dict)

<b> 2. Clean the data </b>

In [210]:
#=================look for row location and use df.drop to remove them =====================
i=0
idx=[]
for b in df['Borough']=='Not assigned':
    if b==True:
        idx.append(i)
    i=i+1
df.drop(index=idx, axis = 0, inplace = True)

#=================look for "Not assigned " in "Neighbourhood" and assign them as the "Borough" value=====================
i=0
idx=[]
for b in df['Neighbourhood']=="Not assigned":
    if b==True:
        df.replace(df.iloc[i,2],df.iloc[i,1],inplace=True)
    i=i+1

#===============look for duplicated value in column "Postcode"====================
df.sort_values(by=['Postcode'],inplace=True)
df.reset_index(drop=True, inplace = True)
df_fin=pd.DataFrame({'Postcode':[],'Borough':[],'Neighbourhood':[]})
df_fin.reset_index(drop=True,inplace = True)
df_fin.append(df.iloc[[0,1]],sort = False)
#for i in range(1, len(df)):
i =0
b=i+1
while b<len(df['Postcode']): 
    if df.iloc[i,0]==df.iloc[b,0]:
        df.iloc[i,2]=df.iloc[i,2]+", " + df.iloc[b,2]
        df.drop([b,b],inplace =True)
        df.reset_index(drop=True, inplace = True)
    i=i+1
    b=i+1

<b> print the number of rows in the df </b>

In [208]:
df.shape

(210, 3)